In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('/content/ai_city.xlsx')


In [ ]:
data


city      lat      lng
0            Delhi  28.6100  77.2300
1           Mumbai  19.0761  72.8775
2          Kolkāta  22.5675  88.3700
3        Bangalore  12.9789  77.5917
4          Chennai  13.0825  80.2750
5        Hyderābād  17.3850  78.4867
6             Pune  18.5203  73.8567
7        Ahmedabad  23.0300  72.5800
8            Sūrat  21.1702  72.8311
9             Bāli  25.4810  85.2227
10         Lucknow  26.8500  80.9500
11          Jaipur  26.9000  75.8000
12          Manali  13.1667  80.2667
13          Shimla  31.1033  77.1722
14          Nāgpur  21.1497  79.0806
15    Secunderābād  17.4399  78.4983
16        Vadodara  22.3000  73.2000
17  Vishākhapatnam  17.7042  83.2978
18          Indore  22.7167  75.8472
19      Puducherry  11.9167  79.8167
20          Bhopāl  23.2500  77.4167
21      Kūkatpalli  17.4849  78.4138
22           Patna  25.6000  85.1000
23            Kota  25.1800  75.8300
24       New Delhi  28.6139  77.2090
25          Raipur  21.2500  81.6300
26            Āgra  27.1800  78.0200
27         Madurai   9.9252  78.1198
28      Chandīgarh  30.7500  76.7800
29           Nāsik  20.0000  73.7800
30         Gwalior  26.2215  78.1780
31      Aurangābād  19.8800  75.3200
32        Vārānasi  25.3189  83.0128
33         Jodhpur  26.2800  73.0200
34        Srīnagar  34.0900  74.7900
35        Amritsar  31.6400  74.8600
36        Guwāhāti  26.1722  91.7458
37          Rānchi  23.3600  85.3300

In [ ]:
import pandas as pd
import folium
import networkx as nx
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of Earth in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Read data from Excel file
data = pd.read_excel('/content/ai_city.xlsx')

# User input: Source and destination cities
source_city_name = input("Enter the source city: ")
destination_city_name = input("Enter the destination city: ")

# Find the cities in the DataFrame
source_row = data[data['city'] == source_city_name]
destination_row = data[data['city'] == destination_city_name]

if not source_row.empty and not destination_row.empty:
    # Source and destination cities found, use their coordinates
    source_lat = source_row['lat'].iloc[0]
    source_lon = source_row['lng'].iloc[0]
    destination_lat = destination_row['lat'].iloc[0]
    destination_lon = destination_row['lng'].iloc[0]

    # Calculate distances
    data['distance_to_target'] = data.apply(lambda row: calculate_distance(row['lat'], row['lng'], source_lat, source_lon), axis=1)

    # Display the table in a neat format
    print(data[['city', 'distance_to_target']].to_markdown(index=False))

    # Create a folium map centered around the source city
    m = folium.Map(location=[source_lat, source_lon], zoom_start=6)

    # Add markers for each city
    for _, row in data.iterrows():
        folium.Marker([row['lat'], row['lng']], popup=row['city']).add_to(m)

    # Add the source and destination markers with red color
    folium.Marker([source_lat, source_lon], popup=source_city_name, icon=folium.Icon(color='red')).add_to(m)
    folium.Marker([destination_lat, destination_lon], popup=destination_city_name, icon=folium.Icon(color='red')).add_to(m)

    # Create a graph using NetworkX
    G = nx.Graph()

    # Add nodes and edges to the graph
    for _, row in data.iterrows():
        G.add_node(row['city'], pos=(row['lat'], row['lng']))

    for _, row1 in data.iterrows():
        for _, row2 in data.iterrows():
            if row1['city'] != row2['city']:
                distance = calculate_distance(row1['lat'], row1['lng'], row2['lat'], row2['lng'])
                G.add_edge(row1['city'], row2['city'], weight=distance)

    # Find the minimum spanning tree using Prim's algorithm
    mst = nx.minimum_spanning_tree(G)

    # Add the edges of the MST to the map in green
    for edge in mst.edges():
        city1, city2 = edge
        city1_coordinates = (data[data['city'] == city1]['lat'].iloc[0], data[data['city'] == city1]['lng'].iloc[0])
        city2_coordinates = (data[data['city'] == city2]['lat'].iloc[0], data[data['city'] == city2]['lng'].iloc[0])
        folium.PolyLine([city1_coordinates, city2_coordinates], color="green", weight=2.5, opacity=1).add_to(m)
    def calculate_distance_heuristic(node1, node2):
      lat1, lon1 = G.nodes[node1]['pos']
      lat2, lon2 = G.nodes[node2]['pos']
    # Calculate the heuristic distance using Haversine formula
      R = 6373.0  # approximate radius of Earth in km
      dlat = radians(lat2 - lat1)
      dlon = radians(lon2 - lon1)
      a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
      c = 2 * atan2(sqrt(a), sqrt(1 - a))
      distance = R * c
      return distance

    # Convert the MST into a NetworkX graph
    mst_graph = nx.Graph(mst)

    # Find the shortest path using A* algorithm
    shortest_path = nx.astar_path(mst_graph, source_city_name, destination_city_name, heuristic=calculate_distance_heuristic)


    print("Shortest path on the MST:", shortest_path)

    # Save the map as an HTML file
    map_html_path = 'map_with_mst.html'
    m.save(map_html_path)
    print(f"Map with minimum spanning tree saved as '{map_html_path}'. Open the file in a web browser to view it.")
else:
    print(f"One or both of the cities '{source_city_name}' and '{destination_city_name}' not found in the data.")


Enter the source city: Delhi
Enter the destination city: Mumbai
| city           |   distance_to_target |
|:---------------|---------------------:|
| Delhi          |              0       |
| Mumbai         |           1148.87    |
| Kolkāta        |           1303.07    |
| Bangalore      |           1739.05    |
| Chennai        |           1755.63    |
| Hyderābād      |           1255.14    |
| Pune           |           1173.58    |
| Ahmedabad      |            775.677   |
| Sūrat          |            938.789   |
| Bāli           |            864.64    |
| Lucknow        |            415.254   |
| Jaipur         |            236.616   |
| Manali         |           1746.25    |
| Shimla         |            277.385   |
| Nāgpur         |            850.512   |
| Secunderābād   |           1249.18    |
| Vadodara       |            810.02    |
| Vishākhapatnam |           1361.88    |
| Indore         |            669.989   |
| Puducherry     |           1876.11    |
| Bhopāl    